# 💽 Lecture 5 (Part 2, Data Storage Types) – Data 100, Spring 2025

Data 100, Spring 2025

[Acknowledgments Page](https://ds100.org/sp25/acks/)

In [21]:
import numpy as np
import pandas as pd

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# Use 2 decimal places instead of scientific notation in pandas
pd.set_option('display.float_format', '{:.2f}'.format)

# Silence some spurious seaborn warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


<br><br><br>

---

## 🤹‍♀️ File Formats Other than CSV

There are many file types for storing structured data: CSV, TSV, JSON, XML, ASCII, SAS...
* In lecture, we will cover TSV and JSON since `pandas` supports them out-of-box.

TSV stands for tab-separated values. Think CSV, but with tabs (`\t`) as the delimeter instead of commas.

- TSVs are [technically faster](https://stackoverflow.com/questions/11130120/choosing-between-tsv-and-csv) to import than CSVs. 

- But, historical tradition has led to CSV being the more popular file format.

<br>

Like we did with `data/cdc_tuberculosis.csv`, you can open `data/cdc_tuberculosis.tsv` in the JupyterLab Explorer to see its contents.

<br>
<br>

The `pd.read_csv` function also reads TSVs if we specify the **delimiter** with parameter `sep='\t'` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)).

In [ ]:
# Identical data to Part 1!
tuberculosis_df_tsv = pd.read_csv("data/cdc_tuberculosis.tsv", sep='\t')
tuberculosis_df_tsv.head()

In [ ]:
# CSV representation
with open("data/cdc_tuberculosis.csv", "r") as f:
    for _, row in zip(range(4), f):
        print(row) # print raw strings

In [ ]:
# TSV representation
with open("data/cdc_tuberculosis.tsv", "r") as f:
    for _, row in zip(range(4), f):
        print(row) # print raw strings

In [ ]:
# Print literal \t instead of tabbed spaces:
with open("data/cdc_tuberculosis.tsv", "r") as f:
    for _, row in zip(range(4), f):
        print(repr(row)) # print raw strings

<br><br>

**Instructor note: Return to Slides!**


<br> <br>

---


### 🪆 JSON (JavaScript Object Notation)

The [congress.gov API](https://gpo.congress.gov/#/) (Application Programming Interface) provides data about the activities and members of the United States Congress (i.e., the House of Representatives and the Senate).

- Click the link above to see the kinds of information provided by the API. 

<br>

To get a JSON file containing information about the current members of Congress from California, you could use the following **API call**:

- `https://api.congress.gov/v3/member/CA?api_key=[INSERT_KEY]&limit=250&format=json&currentMember=True`

- You can instantly sign up for a congress.gov **API key** [here](https://gpo.congress.gov/sign-up/). Once you have your key, replace `[INSERT_KEY]` above with your key, and enter the API call as a URL in your browser. What happens? 

- Once the JSON from the API call is visible in your browser, you can click `File` --> `Save Page As` to save the JSON file to your coputer.

- Coarsely, API keys are used to track how much a given user engages with the API. There might be limits to the number of API calls (e.g., congress.gov API limits to 5,000 calls per hour), or a cost for API calls (e.g., using the OpenAI API for programmatically using ChatGPT).

<br>

For convenience, the JSON file from the call above has already been downloaded for you and is saved at `data/ca-congress-members.json`. 

#### 📁 File contents

Let's look at `data/ca-congress-members.json` in the JupyterLab Explorer.

- Right-click the file, and then click `Open With` --> `Editor`.

- You'll notice that JSON looks a lot like a Python dictionary.

- Berkeley, CA is in the 12th district. Can you find our representative in Congress?

> Note: In general, it's a good idea to check the file size before opening a file in JupyterLab. Very large files can cause crashes. See `os.path.getsize` [documentation](https://docs.python.org/3/library/os.path.html#os.path.getsize).

We can programmatically view the first couple lines of the file using the same functions we used with CSVs:

In [ ]:
congress_file = "data/ca-congress-members.json"

# Inspect the first five lines of the file
with open(congress_file, "r") as f:
    for i, row in enumerate(f):
        print(row)
        if i >= 4: break

### 🐍 EDA: Digging into JSON with Python

JSON data closely matches the internal Python object model.  

- In the following cell, we import the entire JSON datafile into a Python dictionary using the `json` package.

In [ ]:
import json

# Import the JSON file into Python as a dictionary
with open(congress_file, "rb") as f:
    congress_json = json.load(f)

type(congress_json)

The `congress_json` variable is a dictionary encoding the data in the JSON file.

Below, we access the first element of the `members` element of the `congress_json` dictionary.

- This first element is also a dictionary (and there are more dictionaries inside of it!)

In [ ]:
# Grab the list corresponding to the `members` key in the JSON dictionary, 
# and then grab the first element of this list.
# In a moment, we'll see how we knew to use the key `members`, and that
# the resulting object is a list.
congress_json['members'][0]

How should we probe a nested dictionary like `congress_json`?

We can start by identifying the top-level **keys** of the dictionary:

In [ ]:
# Grab the top-level keys of the JSON dictionary
congress_json.keys()

Looks like we have three top-level keys: `members`, `pagination`, and `request`.

> You'll often see a top-level `meta` key in JSON files. This does not refer to Meta (formerly Facebook). Instead, it typically refers to metadata (data about the data).  Metadata are often maintained alongside the data.

Let's check the type of the `members` element:

In [ ]:
type(congress_json['members'])

Looks like a list! What are the first two elements?

In [ ]:
congress_json['members'][:2]

More dictionaries! You can repeat the process above to traverse the nested dictionary.

You'll notice that each record of `congress_json['members']` looks like it could be a column of a DataFrame.

- The keys look a lot like column names, and the values could be the entries in each row.

<br> 

But, the two other elements of `congress_json` don't have the same structure as `congress_json['members']`.

- So, they probably don't belong in a DataFrame containing the members of Congress from CA.

- We'll see the implications of this inconsistency in the next section.

In [ ]:
print(congress_json['pagination'])
print(congress_json['request'])

### 🐼 JSON with pandas

`pandas` has a built in function called `pd.read_json` for reading in JSON files. 

- Uncomment the code below and see what happens.

In [110]:
# pd.read_json(congress_file)

Uh oh. Using the default parameters, we got an error. 

- The code above tries to import the entire JSON file located at `congress_file` (`congress_json`), including `congress_json['pagination']` and `congress_json['request']`.

- We only want to make a DataFrame out of `congress_json['members']`.

This time, let's try converting the `members` element of `congress_json` to a DataFrame by using `pd.DataFrame`:

In [ ]:
# Convert dictionary to DataFrame
congress_df = pd.DataFrame(congress_json['members'])
congress_df.head()

We've successfully begun to rectangularize our JSON data!

<br><br><br>

**Instructor Note: Return to Slides!**

<br/>

---


## 🕰️ Temporality

Let's briefly look at how we can use pandas `dt` accessors to work with dates/times in a dataset.

We will use the Berkeley Police Department (PD) Calls for Service dataset.

In [ ]:
calls = pd.read_csv("data/Berkeley_PD_-_Calls_for_Service.csv")
calls.head()

Looks like there are three columns with dates/times: `EVENTDT`, `EVENTTM`, and `InDbDate`. 

- `EVENTDT` stands for the **date** when the event took place

- `EVENTTM` stands for the **time of day** the event took place (in 24-hr format)

- `InDbDate` is the date this call is entered into the database.

We can convert these string columns to `datetime` objects using the `pd.to_datetime` function.

In [ ]:
# pd.to_datetime() is smart -- It can often infer what you want based on
# the format of the datetime string.
# But, it's not always right! It's good practice to specify the format of 
# your datetimes. See the documentation and the `format` argument.

# Without format specified:
calls["EVENTDT"] = pd.to_datetime(calls["EVENTDT"])

# With format specified:
# calls["EVENTDT"] = pd.to_datetime(calls["EVENTDT"], format='%m/%d/%Y %I:%M:%S %p')

calls.head()

Now we can use the `dt` accessor on this column.

We can get the month:

In [ ]:
# 1 - January, 2 - February, ..., 12 - December
calls["EVENTDT"].dt.month

Which day of the week the date is on:

In [ ]:
# 0 - Monday, 1 - Tuesday, ..., 6 - Sunday
calls["EVENTDT"].dt.dayofweek

We can also sort by datetime:

In [ ]:
# Sort the DataFrame by datetime to find the earliest call.
calls.sort_values("EVENTDT").head()

We can also do many things with the `dt` accessor like switching time zones and converting time back to UNIX/POSIX time. Check out the documentation on [`.dt` accessor](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors) and [time series/date functionality](https://pandas.pydata.org/docs/user_guide/timeseries.html#).

What type are datetime objects?

In [ ]:
calls["EVENTDT"].dtype

`ns` above stands for nanoseconds.

- `<M8` refers to the Numpy type `datetime64`

Under the hood, datetimes in Pandas are integers representing the number of **nanoseconds** since 1/1/1970 UTC.

In [ ]:
# datetimes in pandas are stored as integers representing number of 
# nanoseconds since 1970-01-01
calls["EVENTDT"].astype(int)

<br><br><br>

**Instructor Note: Return to Slides!**

<br/>

---


## 🤷 Faithfulness and missing values

To conclude, let's **very** briefly explore missingness in the Berkeley PD Calls for Service dataset.

Looking at the top of the dataframe, we can already see that there are missing values in the `BLKADDR` column.

In [ ]:
calls.head()

We can use the `.isna()` method to get a sense of how often values in `BLKADDR` are missing.

- The `.isnull()` method is functionally equivalent. 

In [ ]:
# isna() returns a Series of booleans indicating whether each element 
# # in the Series is missing.
print(calls['BLKADDR'].isna().head())

# The mean of a Series of booleans is the proportion of booleans that are True.
calls['BLKADDR'].isna().mean()

It looks like missing values are actually quite rare: Only 0.8% of records are missing a value in `BLKADDR`.

Why are these values missing? 

- Again, looking at just the first few rows, we see that `NaN` values in `BLKADDR` appear to be accompanied by latitude/longitude coordinates in the `Block_Location` column.

- In all likelihood, missing values in `BLKADDR` probably correspond to locations that do not have a defined address in the officer's navigation or GPS system.

<br>

The best default approach here: Leave the rows with missing `BLKADDR` untouched, or replace the `NaN` values with a `MISSING` indicator.

- In the future, if we wanted to conduct an analysis of the streets where police incidents were most common, we might impute `BLKADDR` by using the nearest street, which we could identify with an external package.

<br>

For a very rough sense of missingness in each column of a DataFrame, you can use the `info()` method.

- Based on the output, it looks like the only column with missing values is `BLKADDR`.

In [ ]:
# You can see the total number of rows at the top of the .info() output.
# Compare this to the number of non-null values in the BLKADDR column.
calls.info()

<br><br><br>

**Instructor Note: Return to Slides!**